In [1]:
import os
import numpy as np
import torch
from PIL import Image

In [2]:
class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root 
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))
        
    def __getitem__(self, index):
        img_path = os.path.join(self.root, "PNGImages", self.imgs[index])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[index])
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)
        mask = np.array(mask)
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]
        masks = mask == obj_ids[:, None, None]
        
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.nonzero(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objs), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        
        image_id = torch.tensor([index])
        area = (boxes[:,3]-boxes[:,1]) * (boxes[:,2]- boxes[:,0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        return img, target
    
    def __len__(self):
        return len(self.imgs)


In [3]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [4]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [8]:
import utils

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

dataset = PennFudanDataset('../QuickStart/data/PennFudanPed/', get_transform(train=True))
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle = True, collate_fn=utils.collate_fn
)
images, targets = next(iter(dataloader))
images = list(image for image in images)
targets = [{k:v for k,v in t.items()} for t in targets]
output = model(images, targets)
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3,500,400)]
predictions = model(x)

In [9]:
output

{'loss_classifier': tensor(0.1405, grad_fn=<NllLossBackward0>),
 'loss_box_reg': tensor(0.0762, grad_fn=<DivBackward0>),
 'loss_objectness': tensor(0.0179, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'loss_rpn_box_reg': tensor(0.0060, grad_fn=<DivBackward0>)}

In [10]:
predictions

[{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>),
  'labels': tensor([], dtype=torch.int64),
  'scores': tensor([], grad_fn=<IndexBackward0>)},
 {'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>),
  'labels': tensor([], dtype=torch.int64),
  'scores': tensor([], grad_fn=<IndexBackward0>)}]

In [29]:
from engine import train_one_epoch, evaluate

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_classes = 2
dataset = PennFudanDataset('../QuickStart/data/PennFudanPed/', get_transform(train=True)) 
dataset_test = PennFudanDataset('../QuickStart/data/PennFudanPed/', get_transform(train=False))

indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=utils.collate_fn)   
dataloader_test =  torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=utils.collate_fn)   

model = get_model_instance_segmentation(num_classes)

model.to(device=device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params=params, lr=0.005, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

num_epochs = 10

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, dataloader, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, dataloader_test, device=device)
print("That's it!")

Epoch: [0]  [ 0/60]  eta: 0:16:16  lr: 0.000090  loss: 3.4985 (3.4985)  loss_classifier: 0.6362 (0.6362)  loss_box_reg: 0.1134 (0.1134)  loss_mask: 2.7403 (2.7403)  loss_objectness: 0.0070 (0.0070)  loss_rpn_box_reg: 0.0016 (0.0016)  time: 16.2814  data: 0.0836
Epoch: [0]  [10/60]  eta: 0:13:43  lr: 0.000936  loss: 1.6767 (2.1176)  loss_classifier: 0.4190 (0.4519)  loss_box_reg: 0.3056 (0.3339)  loss_mask: 0.8795 (1.3078)  loss_objectness: 0.0172 (0.0177)  loss_rpn_box_reg: 0.0068 (0.0062)  time: 16.4687  data: 0.0550
Epoch: [0]  [20/60]  eta: 0:10:37  lr: 0.001783  loss: 1.1200 (1.4415)  loss_classifier: 0.2166 (0.3114)  loss_box_reg: 0.2716 (0.3012)  loss_mask: 0.3623 (0.8056)  loss_objectness: 0.0172 (0.0172)  loss_rpn_box_reg: 0.0060 (0.0061)  time: 15.9084  data: 0.0431
Epoch: [0]  [30/60]  eta: 0:08:25  lr: 0.002629  loss: 0.6105 (1.1598)  loss_classifier: 0.0934 (0.2369)  loss_box_reg: 0.2320 (0.2838)  loss_mask: 0.2182 (0.6178)  loss_objectness: 0.0065 (0.0144)  loss_rpn_box_re

KeyboardInterrupt: 